## https://public.roboflow.com/object-detection/aquarium

In [1]:
%pip install -q roboflow
from roboflow import Roboflow

rf = Roboflow(model_format="yolov5", notebook="ultralytics")

     |████████████████████████████████| 41 kB 712 kB/s 
     |████████████████████████████████| 67 kB 6.7 MB/s 
     |████████████████████████████████| 138 kB 66.6 MB/s 
     |████████████████████████████████| 54 kB 3.5 MB/s 
     |████████████████████████████████| 145 kB 70.8 MB/s 
     |████████████████████████████████| 178 kB 69.8 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 
upload and label your dataset, and get an API KEY here: https://app.roboflow.com/?model=yolov5&ref=ultralytics


In [4]:
rf = Roboflow(api_key="****")
project = rf.workspace().project("aquarium-6gsbl")
dataset = project.version(1).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Aquarium-Combined-1 in yolov5pytorch:: 100%|██████████| 1287/1287 [00:02<00:00, 513.11it/s]


In [5]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 15110, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 15110 (delta 7), reused 6 (delta 1), pack-reused 15088
Receiving objects: 100% (15110/15110), 13.73 MiB | 31.81 MiB/s, done.
Resolving deltas: 100% (10440/10440), done.
/content/yolov5
     |████████████████████████████████| 1.6 MB 35.0 MB/s 


In [6]:
!python train.py --img 416 --batch 16 --epochs 100 --data Aquarium-Combined-1/data.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=Aquarium-Combined-1/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-251-g241d798 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_

In [8]:
train_n = ''

!python detect.py --weights runs/train/exp{train_n}/weights/best.pt --img 416 --save-txt --conf 0.5 --source Aquarium-Combined-1/test/images

detect: weights=['runs/train/exp/weights/best.pt'], source=Aquarium-Combined-1/test/images, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-251-g241d798 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 157 layers, 7029004 parameters, 0 gradients, 15.8 GFLOPs
image 1/63 /content/yolov5/Aquarium-Combined-1/test/images/IMG_2289_jpeg_jpg.rf.3685b6f97989ea52dbe7a411ba4440f5.jpg: 416x320 1 puffin, 11.2ms
image 2/63 /content/yolov5/Aquarium-Combined-1/test/images/IMG_2301_jpeg_jpg.rf.25b91e7c1edea2c699158c7033636352.jpg: 416x320 13 penguins, 7.9ms
image 3/63 /content/yolov5/Aquarium-Combined-1/t

In [9]:
classes = {0: 'fish',
           1: 'jellyfish',
           2: 'penguin',
           3: 'puffin',
           4: 'shark',
           5: 'starfish',
           6: 'stingray'}

In [11]:
import pandas as pd
import os

df = pd.DataFrame(columns=["objects"])

det_n = '2'
dir = f"runs/detect/exp{det_n}/labels"
for filename in os.listdir(dir):
  with open(os.path.join(dir, filename), "r") as f:
    text = f.readlines()
    objects = []
    for i in text:
      label = int(i.split(' ')[0])
      objects.append(classes[label])
      index = filename[:-3]+"jpg"
      df.loc[index, 'objects'] = objects

df.head()


,objects
IMG_8599_MOV-3_jpg.rf.6e45d09d7a960ae18ec1eae881a000c9.jpg,"[jellyfish, jellyfish, jellyfish, jellyfish]"
IMG_2630_jpeg_jpg.rf.f34b0a20c9a94d1da2d19fb1ec1b86af.jpg,"[fish, fish, stingray]"
IMG_2434_jpeg_jpg.rf.6388f84491e3a91b442c85f3f4c326c3.jpg,"[shark, fish, fish]"
IMG_8404_jpg.rf.3043f4aaeabff913d5fe30286359a8a4.jpg,"[fish, shark, fish, fish, fish, fish, fish, fi..."
IMG_3173_jpeg_jpg.rf.e7f25511eb5e01f409939583c4b8d025.jpg,"[penguin, penguin, penguin, penguin, penguin, ..."


In [12]:
df.to_csv("/content/result.csv")

In [13]:
def encode(image, objects):
  for x in objects:
    data.loc[image, x] = True

data = pd.DataFrame(index = df.index, columns = classes.values())

for index, row in df.iterrows():
  encode(index, row['objects'])
data = data.fillna(False)

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61 entries, IMG_8599_MOV-3_jpg.rf.6e45d09d7a960ae18ec1eae881a000c9.jpg to IMG_8582_MOV-5_jpg.rf.7c7dc7c7072c2bdcea9db8fdb4c27d75.jpg
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   fish       61 non-null     bool 
 1   jellyfish  61 non-null     bool 
 2   penguin    61 non-null     bool 
 3   puffin     61 non-null     bool 
 4   shark      61 non-null     bool 
 5   starfish   61 non-null     bool 
 6   stingray   61 non-null     bool 
dtypes: bool(7)
memory usage: 2.9+ KB


In [14]:
from mlxtend.frequent_patterns import apriori

min_support = 2/len(data.index)
itemsets = apriori(data, 
                   min_support = min_support,
                   max_len = 2,
                   use_colnames = True)

itemsets.head()

,support,itemsets
0,0.475410,(fish)
1,0.180328,(jellyfish)
2,0.114754,(penguin)
3,0.098361,(puffin)
4,0.213115,(shark)


In [15]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(itemsets, min_threshold=min_support)

rules["antecedents"] = rules["antecedents"].apply(lambda x: list(x)[0])
rules["consequents"] = rules["consequents"].apply(lambda x: list(x)[0])

rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,fish,shark,0.475410,0.213115,0.196721,0.413793,1.941645,0.095404,1.342334
1,shark,fish,0.213115,0.475410,0.196721,0.923077,1.941645,0.095404,6.819672
2,fish,stingray,0.475410,0.163934,0.131148,0.275862,1.682759,0.053212,1.154567
3,stingray,fish,0.163934,0.475410,0.131148,0.800000,1.682759,0.053212,2.622951
4,shark,stingray,0.213115,0.163934,0.049180,0.230769,1.407692,0.014243,1.086885


In [16]:
rels = rules[rules['lift'] > 1]
rels = rels[['antecedents', 'consequents']]

rels.head()

,antecedents,consequents
0,fish,shark
1,shark,fish
2,fish,stingray
3,stingray,fish
4,shark,stingray


In [17]:
!pip install Cython
!pip install owlready2
from owlready2 import *

onto = get_ontology('http://onto.owl')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.5 MB 1.5 MB/s 
  Created wheel for owlready2: filename=Owlready2-0.39-cp37-cp37m-linux_x86_64.whl size=22132061 sha256=d4a06ddf61cc04e33050ad9e2022c41ef9b230b61d272d7f77fc5fcbee5628ad
  Stored in directory: /root/.cache/pip/wheels/c9/5b/fc/da1e42a17f22cd62bfb170f847a3fb541a7f628858ad3595ec
Successfully built owlready2


In [18]:
import types

with onto:
  onto_cl = types.new_class('Piece', (Thing,))
  for i, cl in enumerate(classes.values()):
    inst = onto_cl(f'piece_{i}')
    inst.label = cl

onto.save("/content/onto.owl")

In [19]:
with onto:
  types.new_class('isAssociatedWith', (ObjectProperty, TransitiveProperty))
  for index, rel in rels.iterrows():
    onto_ant = onto.search_one(label=rel['antecedents'])
    onto_con = onto.search_one(label=rel['consequents'])
    onto_ant.isAssociatedWith.append(onto_con)
    
onto.save("/content/onto.owl")

In [20]:
with onto:
  sync_reasoner_pellet(infer_property_values = True)

onto.save('onto_reasonde.owl', format = 'ntriples')


* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /usr/local/lib/python3.7/dist-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/antlr-runtime-3.2.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/xml-apis-1.4.01.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/httpclient-4.2.3.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/commons-codec-1.6.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/aterm-java-1.6.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/log4j-1.2.16.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/jena-iri-0.9.5.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/slf4j-api-1.6.4.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/jgrapht-jdk1.5.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/jena-arq-2.10.0.jar:/usr/local/lib/python3.7/di

* Owlready * Adding relation onto.piece_4 isAssociatedWith onto.piece_4
* Owlready * Adding relation onto.piece_0 isAssociatedWith onto.piece_0
* Owlready * Adding relation onto.piece_6 isAssociatedWith onto.piece_6


* Owlready2 * Pellet took 1.8041620254516602 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)
